In [11]:
# import library
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.feature_selection import SelectKBest, chi2
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression

### Kombinasi 5 :
- Delete duplicate
- Impute missing value using iterative imputer
- Outlier handling with log tranformation
- Encoding
- Standard Scaler
- Feature selection with Decision Tree

In [12]:
df = pd.read_csv('../../Punya Andi/UFC_kombinasi5.csv')
df.head()

,id,weight_class,B_Height_cms,B_Reach_cms,R_win_by_TKO_Doctor_Stoppage,R_Height_cms
0,0.000000,1,5.755616,0,0.000000,5.763622
1,0.693147,3,5.779446,0,0.000000,5.771566
2,1.098612,7,5.817944,0,0.693147,5.817944
3,1.386294,2,5.787265,0,0.000000,5.747544
4,1.609438,3,5.787265,0,0.000000,5.755616


In [13]:
X = df.drop(['B_Reach_cms'], axis=1)
y = df['B_Reach_cms']

In [14]:
scaler = StandardScaler()
X = scaler.fit_transform(X)

In [15]:
from sklearn.model_selection import GridSearchCV
from sklearn.neighbors import KNeighborsRegressor

# Define the parameter grid
param_grid = {
    'n_neighbors': [3, 5, 7, 11, 15],
    'weights': ['uniform', 'distance'],
    'metric': ['euclidean', 'manhattan', 'minkowski'],
    'leaf_size': [10, 20, 30, 40, 50],
    'p': [1, 2]
}

# Create the KNN model
knn = KNeighborsRegressor()

# Create the GridSearchCV object
grid_search = GridSearchCV(knn, param_grid, cv=5, scoring='r2')

# Fit the data to perform grid search
grid_search.fit(X, y)

# Print the best parameters and best score
print("Best Parameters:", grid_search.best_params_)
print("Best R-squared Score:", grid_search.best_score_)


Best Parameters: {'leaf_size': 10, 'metric': 'euclidean', 'n_neighbors': 3, 'p': 1, 'weights': 'uniform'}
Best R-squared Score: 1.0


In [16]:
from sklearn.model_selection import cross_val_score
from sklearn.metrics import mean_squared_error

# Evaluate the best model using 5-fold cross-validation
cv_scores = cross_val_score(grid_search.best_estimator_, X, y, cv=5, scoring='r2')

# Calculate the mean R-squared
mean_r2 = cv_scores.mean()
print("Mean R-squared:", mean_r2)

Mean R-squared: 1.0


In [17]:
# Calculate the RMSE using 5-fold cross-validation
cv_rmse = np.sqrt(np.abs(cross_val_score(grid_search.best_estimator_, X, y, cv=5, scoring='neg_mean_squared_error')))

# Calculate the mean RMSE
mean_rmse = cv_rmse.mean()
print("Mean RMSE:", mean_rmse)

Mean RMSE: 0.0


In [18]:
df_test = pd.read_csv('../../regression_kaggle/UFC_kombinasi5.csv')
df_test = df_test.drop(['B_Reach_cms'], axis=1, errors='ignore')
df_test_id = df_test['id']
df_test = df_test.drop(['id'], axis=1)
df_test = scaler.transform(df_test)
y_pred = grid_search.best_estimator_.predict(df_test)
submission = pd.DataFrame({'id': df_test_id, 'B_Reach_cms': y_pred})
submission.to_csv('pred_kombinasi4_knn.csv', index=False)

ValueError: The feature names should match those that were passed during fit.
Feature names seen at fit time, yet now missing:
- id
